# Emotion Recognition AI - Kaggle Training Notebook

This notebook demonstrates how to train the Emotion Recognition AI model on Kaggle using the RAF-CE dataset.

## Dataset Structure
The dataset should be uploaded to Kaggle with the following structure:
- `/kaggle/input/emotion-images/RAF-AU/aligned/aligned/` - Aligned face images
- `/kaggle/input/emotion-images/RAF-AU/original/original/` - Original face images
- `/kaggle/input/emotions-augmentations/RAFCE_partition.txt` - Train/test/val partition
- `/kaggle/input/emotions-augmentations/RAFCE_emolabel.txt` - Emotion labels
- `/kaggle/input/emotions-augmentations/RAFCE_AUlabel.txt` - Action Unit labels

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install torch torchvision timm tqdm pandas scikit-learn optuna
!pip install transformers peft datasets accelerate

# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## 2. Verify Dataset Paths

In [ ]:
import os

# Define Kaggle paths
paths = {
    'Aligned Images': '/kaggle/input/emotion-images/RAF-AU/aligned/aligned',
    'Original Images': '/kaggle/input/emotion-images/RAF-AU/original/original',
    'Partition File': '/kaggle/input/emotions-augmentations/RAFCE_partition.txt',
    'Emotion Labels': '/kaggle/input/emotions-augmentations/RAFCE_emolabel.txt',
    'AU Labels': '/kaggle/input/emotions-augmentations/RAFCE_AUlabel.txt'
}

# Check if paths exist
print("Dataset Path Verification:")
print("=" * 50)
for name, path in paths.items():
    exists = os.path.exists(path)
    status = "✓" if exists else "✗"
    print(f"{status} {name}: {path}")
    if exists and os.path.isdir(path):
        num_files = len([f for f in os.listdir(path) if f.endswith('.jpg')])
        print(f"  └─ Number of images: {num_files}")

print("=" * 50)

## 3. Load Configuration

In [ ]:
# Import configuration
from config import get_config

# Get configuration
config = get_config()

# Display configuration
print("Configuration Settings:")
print("=" * 60)
print(f"Running on Kaggle: {config['is_kaggle']}")
print(f"Data Root (Aligned): {config['data_root_aligned']}")
print(f"Data Root (Raw): {config['data_root_raw']}")
print(f"Output Directory: {config['output_dir']}")
print(f"Checkpoint Directory: {config['checkpoint_dir']}")
print(f"Log Directory: {config['log_dir']}")
print(f"\nModel Configuration:")
print(f"  - Number of Classes: {config['model_config']['num_classes']}")
print(f"  - Backbone: {config['model_config']['backbone']}")
print(f"  - Pretrained: {config['model_config']['pretrained']}")
print(f"\nTraining Configuration:")
print(f"  - Batch Size: {config['train_config']['batch_size']}")
print(f"  - Learning Rate: {config['train_config']['learning_rate']}")
print(f"  - Epochs: {config['train_config']['num_epochs']}")
print("=" * 60)

## 4. Test Dataset Loading

In [ ]:
from scripts.dataset import RAFCEDataset

# Test loading training dataset
print("Testing Dataset Loading...")
print("=" * 50)

# Load training dataset (partition_id=0)
train_dataset = RAFCEDataset(partition_id=0, use_aligned=True)
print(f"✓ Training dataset loaded: {len(train_dataset)} samples")

# Load validation dataset (partition_id=2)
val_dataset = RAFCEDataset(partition_id=2, use_aligned=True)
print(f"✓ Validation dataset loaded: {len(val_dataset)} samples")

# Load test dataset (partition_id=1)
test_dataset = RAFCEDataset(partition_id=1, use_aligned=True)
print(f"✓ Test dataset loaded: {len(test_dataset)} samples")

# Display a sample
sample = train_dataset[0]
print(f"\nSample Information:")
print(f"  - Image ID: {sample['image_id']}")
print(f"  - Label: {sample['label'].item()}")
print(f"  - AUs: {sample['aus']}")
print(f"  - Image shape: {sample['image'].shape}")
print("=" * 50)

## 5. Visualize Sample Images

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms

# Get emotion labels from config
emotion_labels = config['emotion_labels']

# Create a dataset without transforms for visualization
vis_dataset = RAFCEDataset(partition_id=0, use_aligned=True)

# Display sample images
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i in range(8):
    sample = vis_dataset[i]
    image = sample['image']
    label = sample['label'].item()
    
    # Convert tensor to numpy if needed
    if isinstance(image, torch.Tensor):
        image = image.permute(1, 2, 0).numpy()
        # Denormalize
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = image * std + mean
        image = np.clip(image, 0, 1)
    
    axes[i].imshow(image)
    axes[i].set_title(f"{emotion_labels.get(label, f'Class {label}')}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## 6. Analyze Class Distribution

In [ ]:
import pandas as pd
from collections import Counter

# Get class distribution
train_labels = [train_dataset.emotions[img_id] for img_id in train_dataset.image_ids]
label_counts = Counter(train_labels)

# Create DataFrame for display
df = pd.DataFrame([
    {'Class': k, 'Emotion': emotion_labels.get(k, f'Class {k}'), 'Count': v}
    for k, v in sorted(label_counts.items())
])

print("Class Distribution in Training Set:")
print("=" * 60)
print(df.to_string(index=False))
print("=" * 60)

# Plot distribution
plt.figure(figsize=(12, 6))
plt.bar(range(len(label_counts)), [label_counts[k] for k in sorted(label_counts.keys())])
plt.xticks(range(len(label_counts)), [emotion_labels.get(k, f'Class {k}') for k in sorted(label_counts.keys())], 
           rotation=45, ha='right')
plt.xlabel('Emotion Class')
plt.ylabel('Number of Samples')
plt.title('Class Distribution in Training Set')
plt.tight_layout()
plt.show()

## 7. Train the Model

You can either:
1. Run the training script directly
2. Use the training function in this notebook

### Option 1: Run Training Script

In [ ]:
# Run the training script with default parameters
!python kaggle_train.py --model resnet50 --epochs 50 --batch_size 32 --lr 0.001

### Option 2: Train in Notebook

In [ ]:
# Training parameters
MODEL_TYPE = 'resnet50'  # Options: resnet50, efficientnet_b0, vit_base_patch16_224
NUM_EPOCHS = 50
BATCH_SIZE = 32
LEARNING_RATE = 0.001
USE_AUGMENTATION = True

print(f"Starting training with {MODEL_TYPE} model...")
print(f"Epochs: {NUM_EPOCHS}, Batch Size: {BATCH_SIZE}, LR: {LEARNING_RATE}")

# Import training function
from kaggle_train import train_model

# Create args object
class Args:
    def __init__(self):
        self.model = MODEL_TYPE
        self.epochs = NUM_EPOCHS
        self.batch_size = BATCH_SIZE
        self.lr = LEARNING_RATE
        self.augmentation = USE_AUGMENTATION

args = Args()

# Train the model
model, history = train_model(config, args)

## 8. Plot Training History

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy plot
axes[0].plot(history['train_accs'], label='Train Accuracy', marker='o')
axes[0].plot(history['val_accs'], label='Validation Accuracy', marker='s')
axes[0].axhline(y=history['best_val_acc'], color='r', linestyle='--', 
               label=f"Best Val Acc: {history['best_val_acc']:.2f}%")
axes[0].axvline(x=history['best_epoch']-1, color='g', linestyle='--', 
               label=f"Best Epoch: {history['best_epoch']}")
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy (%)')
axes[0].set_title('Training and Validation Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss plot
axes[1].plot(history['train_losses'], label='Train Loss', marker='o')
axes[1].plot(history['val_losses'], label='Validation Loss', marker='s')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Training and Validation Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print summary
print("\n" + "=" * 60)
print("Training Summary")
print("=" * 60)
print(f"Best Validation Accuracy: {history['best_val_acc']:.2f}%")
print(f"Best Epoch: {history['best_epoch']}")
print(f"Final Validation Accuracy: {history['val_accs'][-1]:.2f}%")
print(f"Final Training Accuracy: {history['train_accs'][-1]:.2f}%")
print("=" * 60)

## 9. Evaluate on Test Set

In [ ]:
from torch.utils.data import DataLoader
from evaluation.metrics import FEREvaluator

# Load best model
checkpoint_path = os.path.join(config['checkpoint_dir'], f"{MODEL_TYPE}_best.pth")
checkpoint = torch.load(checkpoint_path, map_location=device)

# Create model and load weights
model = FERBaseline(model_type=MODEL_TYPE, num_classes=config['model_config']['num_classes']).to(device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"Loaded best model from epoch {checkpoint['epoch']}")
print(f"Validation accuracy: {checkpoint['val_acc']:.2f}%")

# Create test data loader
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_dataset = RAFCEDataset(partition_id=1, transform=val_transform, use_aligned=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

# Evaluate on test set
all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating on test set"):
        images = batch['image'].to(device)
        labels = batch['label'].to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate metrics
evaluator = FEREvaluator()
metrics = evaluator.calculate_metrics(all_labels, all_predictions)

print("\n" + "=" * 60)
print("Test Set Evaluation Results")
print("=" * 60)
print(f"Accuracy: {metrics['accuracy']:.4f}")
print(f"F1 Score (Macro): {metrics['f1_macro']:.4f}")
print(f"F1 Score (Weighted): {metrics['f1_weighted']:.4f}")
print(f"Precision (Macro): {metrics['precision_macro']:.4f}")
print(f"Recall (Macro): {metrics['recall_macro']:.4f}")
print("\nClassification Report:")
print(metrics['full_report'])
print("=" * 60)

## 10. Save Results for Download

In [ ]:
# Create a results summary
results = {
    'model_type': MODEL_TYPE,
    'best_val_accuracy': history['best_val_acc'],
    'best_epoch': history['best_epoch'],
    'final_val_accuracy': history['val_accs'][-1],
    'final_train_accuracy': history['train_accs'][-1],
    'test_accuracy': metrics['accuracy'],
    'test_f1_macro': metrics['f1_macro'],
    'test_f1_weighted': metrics['f1_weighted'],
    'training_args': {
        'epochs': NUM_EPOCHS,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'augmentation': USE_AUGMENTATION
    },
    'training_history': {
        'train_losses': history['train_losses'],
        'val_losses': history['val_losses'],
        'train_accs': history['train_accs'],
        'val_accs': history['val_accs']
    }
}

# Save results
results_path = os.path.join(config['output_dir'], 'training_results.json')
import json
with open(results_path, 'w') as f:
    json.dump(results, f, indent=4)

print(f"Results saved to: {results_path}")
print("\nFiles available for download:")
print("- Best model checkpoint")
print("- Final model checkpoint")
print("- Training history JSON")
print("- Training results JSON")

## 11. Advanced Training Options

### Try Different Models

In [ ]:
# Train with EfficientNet
!python kaggle_train.py --model efficientnet_b0 --epochs 50 --batch_size 32 --lr 0.001

# Train with Vision Transformer
# !python kaggle_train.py --model vit_base_patch16_224 --epochs 50 --batch_size 16 --lr 0.0001

### Hyperparameter Optimization

In [ ]:
# Run hyperparameter optimization (requires more time and GPU)
# This will search for the best hyperparameters
from training.hyperopt import optimize_hyperparameters
from models.baseline import FERBaseline
from torch.utils.data import DataLoader, WeightedRandomSampler
import numpy as np

# Create data loaders
train_dataset = RAFCEDataset(partition_id=0, use_aligned=True)
val_dataset = RAFCEDataset(partition_id=2, use_aligned=True)

# Simple transforms for hyperopt
from torchvision import transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset.transform = train_transform
val_dataset.transform = val_transform

# Calculate weights
targets = [train_dataset.emotions[img_id] for img_id in train_dataset.image_ids]
class_counts = np.bincount(targets)
class_weights = 1.0 / torch.tensor(class_counts, dtype=torch.float)
sample_weights = class_weights[targets]
sampler = WeightedRandomSampler(sample_weights, len(sample_weights))

train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# Run optimization (reduce n_trials for faster results)
best_params, best_value = optimize_hyperparameters(
    FERBaseline,
    train_loader,
    val_loader,
    device,
    n_trials=20,  # Reduce for faster execution
    num_epochs=10  # Reduce for faster execution
)

print(f"\nBest parameters: {best_params}")
print(f"Best validation accuracy: {best_value:.2f}%")

## 12. Conclusion

This notebook provides a complete workflow for training the Emotion Recognition AI model on Kaggle. Key points:

1. **Dataset Setup**: Ensure your dataset is uploaded to Kaggle with the correct structure
2. **Configuration**: The [`config.py`](config.py) file automatically detects Kaggle environment
3. **Training**: Use [`kaggle_train.py`](kaggle_train.py) or train directly in the notebook
4. **Evaluation**: Evaluate on test set and download results
5. **Advanced**: Try different models and hyperparameter optimization

For more advanced features, explore:
- [`training/crossval.py`](training/crossval.py) - Cross-validation
- [`training/distillation.py`](training/distillation.py) - Knowledge distillation
- [`training/hyperopt.py`](training/hyperopt.py) - Hyperparameter optimization
- [`evaluation/xai_enhanced.py`](evaluation/xai_enhanced.py) - Explainable AI

Good luck with your training!